In [41]:
import pandas as pd
import numpy as np

#this constant is the project planned cost. This is an actual from the database
project_cost = 1000



In [42]:
#ingestion 1
df_planned = pd.read_csv('Data/planned.csv',parse_dates=['Start_date','End_date'])
df_planned

,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date
0,1,Activity 1,500,2026-01-03,2026-01-07
1,2,Activity 2,150,2026-01-08,2026-01-09
2,3,Activity 3,250,2026-01-10,2026-01-12
3,4,Activity 4,100,2026-01-11,2026-01-16


In [43]:
df_planned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   SubTaskID     4 non-null      int64         
 1   SubTasks      4 non-null      object        
 2   Planned_Cost  4 non-null      int64         
 3   Start_date    4 non-null      datetime64[ns]
 4   End_date      4 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 292.0+ bytes


In [44]:
#computing for "Activity Weight" 
#formula for activity weight is subtask planned cost / project cost
#the computed values will be in a new column "Activity Weight"
df_planned['Activity_Weight'] = df_planned['Planned_Cost'] / project_cost
df_planned


,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date,Activity_Weight
0,1,Activity 1,500,2026-01-03,2026-01-07,0.50
1,2,Activity 2,150,2026-01-08,2026-01-09,0.15
2,3,Activity 3,250,2026-01-10,2026-01-12,0.25
3,4,Activity 4,100,2026-01-11,2026-01-16,0.10


In [45]:
#Holidays
holidays_2026 = ['2026-01-01', '2026-02-25', '2026-04-09', '2026-05-01', '2026-06-12', '2026-08-31', '2026-11-30', '2026-12-25'] 
weekdays_active = "Mon Tue Wed Thu Fri Sat" #workdays including Saturdays

#creating the busdaycal object for convenient way of checking business days
busDays = np.busdaycalendar(weekmask=weekdays_active, holidays=holidays_2026)  

In [46]:
#adding durations in days
#adding holidays and weekends will affect this computation
#df_planned['Duration_Days'] = (df_planned['End_date'] - df_planned['Start_date']).dt.days + 1

start_d = df_planned["Start_date"].dt.normalize().to_numpy(dtype="datetime64[D]")
end_d   = df_planned["End_date"].dt.normalize().to_numpy(dtype="datetime64[D]")

df_planned["Duration_Days"] = np.busday_count(
    start_d,
    end_d + np.timedelta64(1, 'D'),  # add 1 day to include the end date
    busdaycal=busDays   # optional custom calendar
)
df_planned

,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date,Activity_Weight,Duration_Days
0,1,Activity 1,500,2026-01-03,2026-01-07,0.50,4
1,2,Activity 2,150,2026-01-08,2026-01-09,0.15,2
2,3,Activity 3,250,2026-01-10,2026-01-12,0.25,2
3,4,Activity 4,100,2026-01-11,2026-01-16,0.10,5


In [47]:
#computing the average accomplishment per day
#formula is Activity weight / duration days
df_planned['Average_Accomplishement_per_Day'] = df_planned['Activity_Weight'] / df_planned['Duration_Days']
df_planned

,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date,Activity_Weight,Duration_Days,Average_Accomplishement_per_Day
0,1,Activity 1,500,2026-01-03,2026-01-07,0.50,4,0.125
1,2,Activity 2,150,2026-01-08,2026-01-09,0.15,2,0.075
2,3,Activity 3,250,2026-01-10,2026-01-12,0.25,2,0.125
3,4,Activity 4,100,2026-01-11,2026-01-16,0.10,5,0.020


In [48]:
#creating a shorter dataframe
df_short_planned = df_planned[['SubTaskID','SubTasks','Start_date','End_date','Average_Accomplishement_per_Day']]
df_short_planned

,SubTaskID,SubTasks,Start_date,End_date,Average_Accomplishement_per_Day
0,1,Activity 1,2026-01-03,2026-01-07,0.125
1,2,Activity 2,2026-01-08,2026-01-09,0.075
2,3,Activity 3,2026-01-10,2026-01-12,0.125
3,4,Activity 4,2026-01-11,2026-01-16,0.020


In [49]:
#creating an empty dataframe
df_assembled = pd.DataFrame()

In [50]:
#assembling the dataframe

for index, row in df_short_planned.iterrows():
    df_container = pd.DataFrame() #temporary dataframe to hold generated dates and accomplishments
    print(f"Processing SubTaskID: {row['SubTaskID']}, SubTask: {row['SubTasks']}")
    print(f"{row['SubTasks']}: Start Date: {row['Start_date'].date()}, End Date: {row['End_date'].date()}, Average Accomplishment per Day: {row['Average_Accomplishement_per_Day']:.4f}")
    #generate dates between start and end dates for each activity
    activity_date_range = pd.date_range(start=row['Start_date'], end=row['End_date'], freq='D').tolist()
    df_container['Dates'] = activity_date_range
    df_container['SubTaskID'] = row['SubTaskID']
    df_container['SubTasks'] = row['SubTasks']
    df_container['Average_Accomplishment'] = row['Average_Accomplishement_per_Day']
    df_assembled = pd.concat([df_assembled, df_container], ignore_index=True)

#for notebook viewing only. Remove this in the actual .py file
df_assembled

Processing SubTaskID: 1, SubTask: Activity 1
Activity 1: Start Date: 2026-01-03, End Date: 2026-01-07, Average Accomplishment per Day: 0.1250
Processing SubTaskID: 2, SubTask: Activity 2
Activity 2: Start Date: 2026-01-08, End Date: 2026-01-09, Average Accomplishment per Day: 0.0750
Processing SubTaskID: 3, SubTask: Activity 3
Activity 3: Start Date: 2026-01-10, End Date: 2026-01-12, Average Accomplishment per Day: 0.1250
Processing SubTaskID: 4, SubTask: Activity 4
Activity 4: Start Date: 2026-01-11, End Date: 2026-01-16, Average Accomplishment per Day: 0.0200


,Dates,SubTaskID,SubTasks,Average_Accomplishment
0,2026-01-03,1,Activity 1,0.125
1,2026-01-04,1,Activity 1,0.125
2,2026-01-05,1,Activity 1,0.125
3,2026-01-06,1,Activity 1,0.125
4,2026-01-07,1,Activity 1,0.125
5,2026-01-08,2,Activity 2,0.075
6,2026-01-09,2,Activity 2,0.075
7,2026-01-10,3,Activity 3,0.125
8,2026-01-11,3,Activity 3,0.125
9,2026-01-12,3,Activity 3,0.125


Integrating the bussiness days 

In [51]:
#business day array based on the 'Dates' column of df_assembled

#creating a numpy array of datetime64[D] type from the 'Dates' column
dates_np_array = df_assembled['Dates'].values.astype('datetime64[D]')

#creating the is business day array
busDayArray = np.is_busday(dates_np_array, busdaycal=busDays)

In [52]:
type(dates_np_array)

numpy.ndarray

In [53]:
busDayArray

array([ True, False,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True])

In [54]:
#integrating the business day array into the dataframe

#create the new column 'Is a Business Day'
df_assembled['Is a Business Day'] = busDayArray
df_assembled

,Dates,SubTaskID,SubTasks,Average_Accomplishment,Is a Business Day
0,2026-01-03,1,Activity 1,0.125,True
1,2026-01-04,1,Activity 1,0.125,False
2,2026-01-05,1,Activity 1,0.125,True
3,2026-01-06,1,Activity 1,0.125,True
4,2026-01-07,1,Activity 1,0.125,True
5,2026-01-08,2,Activity 2,0.075,True
6,2026-01-09,2,Activity 2,0.075,True
7,2026-01-10,3,Activity 3,0.125,True
8,2026-01-11,3,Activity 3,0.125,False
9,2026-01-12,3,Activity 3,0.125,True


In [55]:
#rearraning the columns
df_assembled = df_assembled[[
    'Dates',
    'Is a Business Day',
    'SubTaskID',
    'SubTasks',
    'Average_Accomplishment'
]]

df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment
0,2026-01-03,True,1,Activity 1,0.125
1,2026-01-04,False,1,Activity 1,0.125
2,2026-01-05,True,1,Activity 1,0.125
3,2026-01-06,True,1,Activity 1,0.125
4,2026-01-07,True,1,Activity 1,0.125
5,2026-01-08,True,2,Activity 2,0.075
6,2026-01-09,True,2,Activity 2,0.075
7,2026-01-10,True,3,Activity 3,0.125
8,2026-01-11,False,3,Activity 3,0.125
9,2026-01-12,True,3,Activity 3,0.125


In [56]:
# creating a new column where instead of boolean business days values, it will be 1s and 0s
df_assembled = df_assembled.copy()
df_assembled['Is a Business Day_int'] = df_assembled['Is a Business Day'].astype(int)
df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment,Is a Business Day_int
0,2026-01-03,True,1,Activity 1,0.125,1
1,2026-01-04,False,1,Activity 1,0.125,0
2,2026-01-05,True,1,Activity 1,0.125,1
3,2026-01-06,True,1,Activity 1,0.125,1
4,2026-01-07,True,1,Activity 1,0.125,1
5,2026-01-08,True,2,Activity 2,0.075,1
6,2026-01-09,True,2,Activity 2,0.075,1
7,2026-01-10,True,3,Activity 3,0.125,1
8,2026-01-11,False,3,Activity 3,0.125,0
9,2026-01-12,True,3,Activity 3,0.125,1


In [57]:
# Creating a new 'Average Accomplishment' columns where the accomplishment is only counted on business days(if business days is = 1)

df_assembled['Average_Accomplishment_Business_Days'] = df_assembled['Average_Accomplishment'] * df_assembled['Is a Business Day_int']
df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment,Is a Business Day_int,Average_Accomplishment_Business_Days
0,2026-01-03,True,1,Activity 1,0.125,1,0.125
1,2026-01-04,False,1,Activity 1,0.125,0,0.000
2,2026-01-05,True,1,Activity 1,0.125,1,0.125
3,2026-01-06,True,1,Activity 1,0.125,1,0.125
4,2026-01-07,True,1,Activity 1,0.125,1,0.125
5,2026-01-08,True,2,Activity 2,0.075,1,0.075
6,2026-01-09,True,2,Activity 2,0.075,1,0.075
7,2026-01-10,True,3,Activity 3,0.125,1,0.125
8,2026-01-11,False,3,Activity 3,0.125,0,0.000
9,2026-01-12,True,3,Activity 3,0.125,1,0.125


In [58]:
#dropping columns 'Average_Accomplishment' and 'Is a Business Day_int' as they are no longer needed
df_assembled = df_assembled.drop(columns=['Average_Accomplishment', 'Is a Business Day_int'])
df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment_Business_Days
0,2026-01-03,True,1,Activity 1,0.125
1,2026-01-04,False,1,Activity 1,0.000
2,2026-01-05,True,1,Activity 1,0.125
3,2026-01-06,True,1,Activity 1,0.125
4,2026-01-07,True,1,Activity 1,0.125
5,2026-01-08,True,2,Activity 2,0.075
6,2026-01-09,True,2,Activity 2,0.075
7,2026-01-10,True,3,Activity 3,0.125
8,2026-01-11,False,3,Activity 3,0.000
9,2026-01-12,True,3,Activity 3,0.125


In [59]:
#replacing zero values on column 'Average_Accomplishments_Business_Days' with NaN for better visualization in the s-curve graph
df_assembled['Average_Accomplishment_Business_Days'] = df_assembled['Average_Accomplishment_Business_Days'].replace(0, np.nan)
df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment_Business_Days
0,2026-01-03,True,1,Activity 1,0.125
1,2026-01-04,False,1,Activity 1,NaN
2,2026-01-05,True,1,Activity 1,0.125
3,2026-01-06,True,1,Activity 1,0.125
4,2026-01-07,True,1,Activity 1,0.125
5,2026-01-08,True,2,Activity 2,0.075
6,2026-01-09,True,2,Activity 2,0.075
7,2026-01-10,True,3,Activity 3,0.125
8,2026-01-11,False,3,Activity 3,NaN
9,2026-01-12,True,3,Activity 3,0.125


In [60]:
#perfoming  aggreagation on Dates column

df_assembled = df_assembled.groupby('Dates')['Average_Accomplishment_Business_Days'].sum().reset_index()
df_assembled

,Dates,Average_Accomplishment_Business_Days
0,2026-01-03,0.125
1,2026-01-04,0.000
2,2026-01-05,0.125
3,2026-01-06,0.125
4,2026-01-07,0.125
5,2026-01-08,0.075
6,2026-01-09,0.075
7,2026-01-10,0.125
8,2026-01-11,0.000
9,2026-01-12,0.145


In [61]:
#exporting df_assembled dataframe to a csv file
df_assembled.to_csv('Output/assembled_data.csv', index=False)

In [62]:
#exporting df_assembled to a json file
df_assembled.to_json('Output/assembled_data.json', orient='records', date_format='iso')


In [63]:
#exporting df_assembled to ndjson file
df_assembled.to_json('Output/assembled_data.ndjson', orient='records', date_format='iso', lines=True)